**THIS FILE CONTAINS THE CODE FOR PREDICTIONS ON THE FINAL TEST SET USING THE XGBOOST REGRESSOR.**

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm

In [ ]:
file_1=open('/content/drive/MyDrive/Project Energy Consumption/df_te_red_final_modified.txt','rb')
df_tr_red_final=pickle.load(file_1)

**DROPPING THE FEATURES WHICH WERE NOT USED FOR THE TRAINING PROCESS**

In [ ]:
df_tr_red_final.drop(['cloud_coverage','sea_level_pressure','wind_direction','wind_speed',
                      'is_summer_month','is_pub_holiday'],axis=1,inplace=True)

In [ ]:
df_tr_red_final.drop('row_id',axis=1,inplace=True)

In [ ]:
file_2=open('/content/drive/MyDrive/Project Energy Consumption/xgb_model_modified.txt','rb')
xgb_model=pickle.load(file_2)

[18:39:10] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


**HERE WE ARE PREDICTING ON THE TEST SET AND THEN USING EXPM1 TO GET THE ORIGINAL METER READINGS FOR THE FINAL PREDICTIONS**

In [ ]:
y_test=np.expm1(xgb_model.predict(df_tr_red_final))

In [ ]:
y_test_round=np.round(y_test,4)

In [ ]:
y_test_df=pd.DataFrame(y_test_round)

In [ ]:
y_test_df.rename(columns={0:'meter_reading'},inplace=True)

In [ ]:
y_test_df['row_id']=y_test_df.index

In [ ]:
y_test_df=y_test_df[['row_id','meter_reading']]

In [ ]:
y_test_df['row_id']=y_test_df['row_id'].astype(int)

**ALL THE  METER READINGS WHICH ARE PREDICTED TO BE LESS THAN ZERO ARE SET TO ZERO AS METER READINGS CANNOT BE ZERO.**

In [ ]:
for i in tqdm(y_test_df[y_test_df['meter_reading']<0].index):
  y_test_df['meter_reading'][i]=0

  0%|          | 0/416791 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
100%|██████████| 416791/416791 [00:34<00:00, 12151.93it/s]


**CONVERTING THE TEST FILE INTO CSV FORMAT FOR SUBMISSION**

In [ ]:
y_test_df.to_csv('xgb.csv',index=False,header=True)

**FINAL PREDICTED METER READINGS BY THE XGBOOST MODEL**

In [ ]:
y_test_df

,row_id,meter_reading
0,0,180.750198
1,1,68.675797
2,2,6.639400
3,3,336.895203
4,4,2155.841553
...,...,...
41697595,41697595,5.609200
41697596,41697596,5.929800
41697597,41697597,4.128000
41697598,41697598,178.042603
